## Машинное обучение для текстов

[yandex_practicum]('https://practicum.yandex.ru/learn/data-scientist/courses/aa114b34-448c-498c-a573-c0eee79180a0/sprints/176096/topics/2cf7a200-3b19-4ad8-aff0-d94a9de8a5ef/lessons/4d8665ee-de3a-4d09-bb6d-714e1bab0a6b/)

### Векторизация текстов 

#### Лемматизация

Токенизация (англ. tokenization) — разбиение текста на токены: отдельные фразы, слова, символы.

Лемматизация (англ. lemmatization) — приведение слова к начальной форме — лемме.

In [1]:
from pymystem3 import Mystem
m = Mystem()

Installing mystem to /Users/tony_builder/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


In [2]:
m.lemmatize("лемматизируй это")

['лемматизировать', ' ', 'это', '\n']